In [ ]:
%pip install -r requirements.txt

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [7]:
from crewai import Agent, Task, Crew

In [9]:
import os
from utils import get_openai_api_key, pretty_print_result 
from utils import get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## Agents

In [10]:
sales_rep_agent = Agent(
    role="Sales Representative.",
    goal="Identify high-value leads that match our ideal customer profile.",
    backstory=("As part of the dynamic sales team at CrewAI, your mission is to scour the digital landscape for potential leads. Armed with cutting-edge"
            " and a strategic mindset, you analyse data, trends and interactions to unearth opportunities that others might overlook. Your work is"
            " crucial in paving the way for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    verbose=True
)

In [11]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications.",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, you stand out as a bridge between potential clients and the solutions they need."
        " By creating engaging, personalized messages, you not only inform leads about our offerings but also make them feel seen and heard. Your role"
        " is pivotal in converting interests into actions, guiding leads through the journey from curiosity to commitment. "
    ),
    allow_delegation=False,
    verbose=True
)

## Tools

In [12]:
from crewai_tools import DirectoryReadTool, FileReadTool, SerperDevTool

In [14]:
directory_read_tool = DirectoryReadTool(directory='./instructions')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

In [15]:
# make your custom tools. We can do so via inheritance. 
from crewai_tools import BaseTool

In [16]:
#custom tool. This is same as the directory_read_tool or search_tool made above. 
class SentimentalAnalysisTools(BaseTool):
    name:str="Sentiment Analysis Tool" # Every tool in crewai needs a name and description. 
    description:str="Analyzes the sentiment of text to ensure positive and engaging communication."

    def _run(self, text:str) -> str:
        # Your custom tool goes here and returns the output as a string

        return "positive"


In [17]:
sentiment_analysis_tool = SentimentalAnalysisTools()

## Tasks

In [18]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

In [19]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

## Crew

In [20]:
crew = Crew(
    agents=[sales_rep_agent, lead_sales_rep_agent],
    tasks=[lead_profiling_task, personalized_outreach_task],
    verbose=2,
    memory=True
)

## Input

In [ ]:
inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Andrew Ng",
    "position": "CEO",
    "milestone": "product launch"
}

result = crew.kickoff(inputs=inputs)

In [ ]:
from IPython.display import Markdown
Markdown(result)